In [ ]:
# ===== 경진대회 문제 1-1: 최종 개선 버전 =====

import os
import re
import json
import warnings
from pathlib import Path
from typing import List, Optional, Tuple
warnings.filterwarnings('ignore')

import torch
import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance, ImageFilter
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

# Transformers
from transformers import AutoProcessor, Qwen3VLForConditionalGeneration

# SAM2.1
import sys
sys.path.append('/home/jjs2403/2026_bootcamp_02/models/sam2')
from sam2.sam2_image_predictor import SAM2ImagePredictor
from omegaconf import OmegaConf
from hydra.utils import instantiate

print("="*80)
print("🚀 경진대회 문제 1-1")
print("="*80)
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")


# ===== 1. Seed 고정 =====
def set_seed(seed: int = 42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    print(f"✅ Seed fixed: {seed}")

SEED = 42
set_seed(SEED)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


# ===== 2. 경로 설정 =====
BASE_DIR = Path("/home/jjs2403/2026_bootcamp_02")
DATASET_DIR = BASE_DIR / "dataset/challenge_datasets/challenge1"
MODEL_DIR = BASE_DIR / "models"

OUTPUT_DIR = Path.home() / "challenge1_outputs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Multi-box safety (only for explicit plural queries)
MAX_MULTI_BOXES = 4
NEGATIVE_OVERSEG_BOX_RATIO = 0.95
NEGATIVE_OVERSEG_IMG_RATIO = 0.60
PLURAL_DEBUG_CSV = OUTPUT_DIR / "plural_debug.csv"

print()
print(f"📁 Dataset: {DATASET_DIR}")
print(f"📁 Output: {OUTPUT_DIR}")


# ===== 3. 데이터셋 로딩 =====
print()
print("="*80)
print("📂 데이터셋 로딩 중...")
print("="*80)

test_img_dir = DATASET_DIR / "test"
image_files = sorted(test_img_dir.glob("*.jpg")) + sorted(test_img_dir.glob("*.png"))

test_images = {}
for img_path in tqdm(image_files, desc="이미지 로딩"):
    img_id = img_path.stem
    test_images[img_id] = Image.open(img_path).convert("RGB")

query_csv = DATASET_DIR / "query.csv"
query_df = pd.read_csv(query_csv)

id_col = next((col for col in query_df.columns if col.lower() in ['id', 'image_id', 'filename']), query_df.columns[0])
query_col = next((col for col in query_df.columns if col.lower() in ['query', 'question', 'text']), query_df.columns[1])

queries = {}
for _, row in query_df.iterrows():
    img_id = str(row[id_col]).replace('.jpg', '').replace('.png', '')
    queries[img_id] = row[query_col]

print(f"✅ 이미지: {len(test_images)}장")
print(f"✅ 쿼리: {len(queries)}개")


# ===== 4. 모델 로딩 =====
print()
print("="*80)
print("🤖 MLLM 모델 로딩 중...")
print("="*80)

model_path = MODEL_DIR / "Qwen3-VL-8B-Instruct"
mllm_processor = AutoProcessor.from_pretrained(str(model_path))
mllm_model = Qwen3VLForConditionalGeneration.from_pretrained(
    str(model_path),
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
print(f"✅ MLLM 로딩 완료")

print()
print("="*80)
print("🎯 SAM2.1 모델 로딩 중...")
print("="*80)

checkpoint_path = "/home/jjs2403/2026_bootcamp_02/models/SAM2.1/weights/sam2.1_hiera_large.pt"
checkpoint = torch.load(checkpoint_path, map_location=DEVICE)

config_path = "/home/jjs2403/2026_bootcamp_02/models/SAM2.1/weights/sam2.1_hiera_l.yaml"
cfg = OmegaConf.load(config_path)

sam_model = instantiate(cfg.model, _recursive_=True)
sam_model.load_state_dict(checkpoint["model"])
sam_model = sam_model.to(DEVICE).eval()

sam_predictor = SAM2ImagePredictor(sam_model)
print(f"✅ SAM2.1 로딩 완료")


# ===== 5. BBox 파싱 함수 =====
def parse_bbox_from_text(text: str, img_width: int, img_height: int):
    # 1) JSON 블록 추출
    json_blocks = re.findall(r"\{.*?\}", text, flags=re.DOTALL)
    for blk in json_blocks:
        try:
            obj = json.loads(blk)
            if "bbox_2d" in obj and isinstance(obj["bbox_2d"], list) and len(obj["bbox_2d"]) >= 4:
                bbox = [float(x) for x in obj["bbox_2d"][:4]]
                return clamp_bbox(bbox, img_width, img_height)
        except:
            pass

    # 2) 리스트 형식 [x1,y1,x2,y2]
    m = re.search(r"\[\s*([\d.]+)\s*,\s*([\d.]+)\s*,\s*([\d.]+)\s*,\s*([\d.]+)\s*\]", text)
    if m:
        bbox = [float(m.group(i)) for i in range(1, 5)]
        return clamp_bbox(bbox, img_width, img_height)

    return None


def clamp_bbox(bbox, w, h):
    """BBox를 이미지 크기에 맞게 조정"""
    x1, y1, x2, y2 = bbox
    x1 = int(max(0, min(w - 1, round(x1))))
    y1 = int(max(0, min(h - 1, round(y1))))
    x2 = int(max(0, min(w - 1, round(x2))))
    y2 = int(max(0, min(h - 1, round(y2))))
    if x2 < x1:
        x1, x2 = x2, x1
    if y2 < y1:
        y1, y2 = y2, y1
    return [x1, y1, x2, y2]


def is_plural_query(query: str) -> bool:
    q = str(query).lower() if query is not None else ""
    if re.search(r"\b(all|every|both|each|among|several|multiple|various)\b", q):
        return True
    if re.search(r"\b(two|three|four|five|six|seven|eight|nine|ten)\b", q):
        return True
    plural_nouns = [
        "people", "men", "women", "children", "kids",
        "players", "targets",
        "vehicles", "cars", "bikes", "boats", "trains",
        "dice", "fruits",
        "items", "objects", "boxes",
        "buttons", "ports", "parts", "signs",
        "labels", "texts", "bars", "lines", "cells", "years",
        "devices",
    ]
    return any(re.search(rf"\b{re.escape(k)}\b", q) for k in plural_nouns)


def bbox_iou(box_a: List[int], box_b: List[int]) -> float:
    ax1, ay1, ax2, ay2 = box_a
    bx1, by1, bx2, by2 = box_b

    ix1 = max(ax1, bx1)
    iy1 = max(ay1, by1)
    ix2 = min(ax2, bx2)
    iy2 = min(ay2, by2)

    iw = max(0, ix2 - ix1 + 1)
    ih = max(0, iy2 - iy1 + 1)
    inter = iw * ih

    area_a = max(0, ax2 - ax1 + 1) * max(0, ay2 - ay1 + 1)
    area_b = max(0, bx2 - bx1 + 1) * max(0, by2 - by1 + 1)
    union = area_a + area_b - inter
    if union <= 0:
        return 0.0
    return inter / union


def dedupe_bboxes(bboxes: List[List[int]]) -> List[List[int]]:
    kept: List[List[int]] = []
    for bbox in bboxes:
        if all(bbox_iou(bbox, k) < 0.7 for k in kept):
            kept.append(bbox)
        if len(kept) >= MAX_MULTI_BOXES:
            break
    return kept


def parse_bboxes_from_text(text: str, img_width: int, img_height: int) -> List[List[int]]:
    bboxes: List[List[int]] = []
    json_blocks = re.findall(r"\{.*?\}", text, flags=re.DOTALL)
    for blk in json_blocks:
        try:
            obj = json.loads(blk)
        except Exception:
            continue

        if "bboxes_2d" in obj and isinstance(obj["bboxes_2d"], list):
            for item in obj["bboxes_2d"]:
                if isinstance(item, list) and len(item) >= 4:
                    bbox = [float(x) for x in item[:4]]
                    bboxes.append(clamp_bbox(bbox, img_width, img_height))
            if bboxes:
                return bboxes

        if "bbox_2d" in obj and isinstance(obj["bbox_2d"], list) and len(obj["bbox_2d"]) >= 4:
            bbox = [float(x) for x in obj["bbox_2d"][:4]]
            bboxes.append(clamp_bbox(bbox, img_width, img_height))
            return bboxes

    m_all = re.findall(r"\[\s*([\d.]+)\s*,\s*([\d.]+)\s*,\s*([\d.]+)\s*,\s*([\d.]+)\s*\]", text)
    for m in m_all:
        bbox = [float(m[0]), float(m[1]), float(m[2]), float(m[3])]
        bboxes.append(clamp_bbox(bbox, img_width, img_height))

    return bboxes


# ===== 6. Visual Focus =====
def apply_visual_focus(image: Image.Image, bbox: List[int]) -> Image.Image:
    """Blur + darken background, keep bbox area sharp and bright."""
    x1, y1, x2, y2 = bbox
    w, h = image.size

    x1 = max(0, min(w - 1, x1))
    y1 = max(0, min(h - 1, y1))
    x2 = max(0, min(w - 1, x2))
    y2 = max(0, min(h - 1, y2))
    if x2 < x1:
        x1, x2 = x2, x1
    if y2 < y1:
        y1, y2 = y2, y1

    blurred = image.filter(ImageFilter.GaussianBlur(radius=10))
    darkened = ImageEnhance.Brightness(blurred).enhance(0.5)
    focused = darkened.copy()

    patch = image.crop((x1, y1, x2 + 1, y2 + 1))
    focused.paste(patch, (x1, y1))
    return focused


# ===== 7. BBox 예측 (0.49 프롬프트 적용) =====
@torch.no_grad()
def predict_bbox_with_mllm(
    model,
    processor,
    image: Image.Image,
    query: str,
    plural: bool
) -> List[List[int]]:

    w, h = image.size

    if plural:
        prompt = f"""You are given an image and a text query.
Infer all target objects/regions that best satisfy the query.
Output ONLY one JSON object with key bboxes_2d=[[x1,y1,x2,y2], ...] in pixel coordinates.
No extra text.
Query: {query}
""".strip()
    else:
        # JSON 형식 + single target + 픽셀 좌표
        prompt = f"""You are given an image and a text query.
Infer the single target object/region that best satisfies the query.
Output ONLY one JSON object with key bbox_2d=[x1,y1,x2,y2] in pixel coordinates.
No extra text.
Query: {query}
""".strip()

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": prompt}
            ]
        }
    ]

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(
        text=[text],
        images=[image],
        return_tensors="pt"
    )

    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") if hasattr(v, "to") else v for k, v in inputs.items()}

    output_ids = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False, 
        num_beams=1
    )

    out_text = processor.batch_decode(output_ids, skip_special_tokens=False)[0]
    if plural:
        bboxes = parse_bboxes_from_text(out_text, w, h)
        if not bboxes:
            bboxes = [[0, 0, w - 1, h - 1]]
        return dedupe_bboxes(bboxes)

    bbox = parse_bbox_from_text(out_text, w, h)
    if bbox is None:
        bbox = [0, 0, w - 1, h - 1]
    return [bbox]


@torch.no_grad()
def predict_bbox_refine(
    model,
    processor,
    image: Image.Image,
    query: str
) -> Tuple[List[List[int]], bool]:
    """Two-pass refinement with visual focus (no cropping)."""
    w, h = image.size
    fallback = [0, 0, w - 1, h - 1]
    plural = is_plural_query(query)

    bboxes_coarse = predict_bbox_with_mllm(model, processor, image, query, plural)
    used_fallback = (len(bboxes_coarse) == 1 and bboxes_coarse[0] == fallback)
    if used_fallback:
        return bboxes_coarse, True

    if plural:
        return bboxes_coarse, False

    focused_image = apply_visual_focus(image, bboxes_coarse[0])
    bboxes_refined = predict_bbox_with_mllm(model, processor, focused_image, query, plural)

    if len(bboxes_refined) == 1 and bboxes_refined[0] == fallback:
        return bboxes_coarse, False

    return bboxes_refined, False


# ===== 8. SAM Mask 생성 (multimask) =====
@torch.no_grad()
def generate_mask_with_sam(
    sam_predictor,
    image: Image.Image,
    bbox: List[int],
    use_points: bool = False
) -> np.ndarray:
    """
    multimask_output=True + 최고 선택
    """
    img_array = np.array(image, dtype=np.uint8)
    sam_predictor.set_image(img_array)

    box = np.array(bbox, dtype=np.float32)

    if not use_points:
        masks, scores, _ = sam_predictor.predict(
            box=box[None, :],
            multimask_output=True
        )
        best_idx = int(np.argmax(scores))
        return masks[best_idx].astype(bool)

    cx = (bbox[0] + bbox[2]) / 2.0
    cy = (bbox[1] + bbox[3]) / 2.0
    point_coords = np.array([[cx, cy]], dtype=np.float32)
    point_labels = np.array([1], dtype=np.int32)

    masks, scores, _ = sam_predictor.predict(
        box=box[None, :],
        point_coords=point_coords,
        point_labels=point_labels,
        multimask_output=True
    )

    best_idx = int(np.argmax(scores))
    best_mask = masks[best_idx].astype(bool)

    mask_area = int(best_mask.sum())
    box_area = max(1, (bbox[2] - bbox[0]) * (bbox[3] - bbox[1]))
    img_area = max(1, img_array.shape[0] * img_array.shape[1])
    ratio_box = mask_area / float(box_area)
    ratio_img = mask_area / float(img_area)

    if ratio_box > NEGATIVE_OVERSEG_BOX_RATIO or ratio_img > NEGATIVE_OVERSEG_IMG_RATIO:
        tight_bbox = mask_to_bbox(best_mask)
        if tight_bbox is not None:
            tight_box = np.array(tight_bbox, dtype=np.float32)
            masks_t, scores_t, _ = sam_predictor.predict(
                box=tight_box[None, :],
                point_coords=point_coords,
                point_labels=point_labels,
                multimask_output=True
            )
            best_mask = masks_t[int(np.argmax(scores_t))].astype(bool)
            mask_area = int(best_mask.sum())
            ratio_box = mask_area / float(box_area)
            ratio_img = mask_area / float(img_area)

    if ratio_box > NEGATIVE_OVERSEG_BOX_RATIO or ratio_img > NEGATIVE_OVERSEG_IMG_RATIO:
        neg = select_negative_point(best_mask, bbox)
        if neg is not None:
            point_coords = np.array([[cx, cy], neg], dtype=np.float32)
            point_labels = np.array([1, 0], dtype=np.int32)
            masks2, scores2, _ = sam_predictor.predict(
                box=box[None, :],
                point_coords=point_coords,
                point_labels=point_labels,
                multimask_output=True
            )
            best2 = masks2[int(np.argmax(scores2))].astype(bool)
            return best2

    return best_mask


def select_negative_point(mask: np.ndarray, bbox: List[int]) -> Optional[List[float]]:
    x1, y1, x2, y2 = bbox
    h, w = mask.shape[:2]
    x1 = max(0, min(w - 1, x1))
    y1 = max(0, min(h - 1, y1))
    x2 = max(0, min(w - 1, x2))
    y2 = max(0, min(h - 1, y2))

    bbox_mask = np.zeros_like(mask, dtype=bool)
    bbox_mask[y1 : y2 + 1, x1 : x2 + 1] = True
    candidates = np.argwhere(bbox_mask & ~mask)
    if candidates.size == 0:
        return None
    idx = len(candidates) // 2
    y, x = candidates[idx]
    return [float(x), float(y)]


def mask_to_bbox(mask: np.ndarray) -> Optional[List[int]]:
    ys, xs = np.where(mask)
    if xs.size == 0 or ys.size == 0:
        return None
    x1 = int(xs.min())
    y1 = int(ys.min())
    x2 = int(xs.max())
    y2 = int(ys.max())
    return [x1, y1, x2, y2]


def bbox_to_mask(bbox: List[int], shape: Tuple[int, int]) -> np.ndarray:
    mask = np.zeros(shape, dtype=bool)
    x1, y1, x2, y2 = bbox
    mask[y1 : y2 + 1, x1 : x2 + 1] = True
    return mask


# ===== 9. RLE Encoding =====
def rle_encode(mask):
    """마스크를 RLE 형식으로 인코딩"""
    if torch.is_tensor(mask):
        mask = mask.detach().cpu().numpy()

    pixels = mask.flatten()
    pixels = np.concatenate([[False], pixels, [False]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]

    return ' '.join(str(x) for x in runs)


# ===== 10. 메인 파이프라인 (Visual Focus) =====
print()
print("="*80)
print("📍 BBox 예측 시작 (Visual Focus Refinement)...")
print("="*80)

bbox_dict = {}
fallback_count = 0
plural_debug_rows = []

for img_id in tqdm(sorted(test_images.keys()), desc="BBox 예측"):
    image = test_images[img_id]
    query = queries.get(img_id, "")

    if not query:
        continue

    bboxes, used_fallback = predict_bbox_refine(mllm_model, mllm_processor, image, query)
    bbox_dict[img_id] = bboxes
    if used_fallback:
        fallback_count += 1
    if is_plural_query(query):
        plural_debug_rows.append({
            "id": img_id,
            "query": str(query),
            "num_boxes": len(bboxes),
            "boxes": json.dumps(bboxes),
            "used_fallback": bool(used_fallback),
        })

print()
print(f"✅ BBox 예측 완료: {len(bbox_dict)}/{len(test_images)}")
print(f"   Fallback 사용: {fallback_count}개")
debug_df = pd.DataFrame(plural_debug_rows)
debug_df.to_csv(PLURAL_DEBUG_CSV, index=False)
print(f"   Plural debug saved: {PLURAL_DEBUG_CSV} (rows: {len(debug_df)})")


print()
print("="*80)
print("🎭 Mask 생성 시작 (multimask)...")
print("="*80)

mask_dict = {}

for img_id in tqdm(sorted(bbox_dict.keys()), desc="Mask 생성"):
    image = test_images[img_id]
    query = queries.get(img_id, "")
    bboxes = bbox_dict[img_id]
    plural = is_plural_query(query)

    try:
        union_mask = np.zeros((image.size[1], image.size[0]), dtype=bool)
        img_area = float(image.size[0] * image.size[1])

        for bbox in bboxes:
            mask = generate_mask_with_sam(sam_predictor, image, bbox, use_points=plural)
            box_area = max(1, (bbox[2] - bbox[0]) * (bbox[3] - bbox[1]))
            mask_area = int(mask.sum())
            ratio_box = mask_area / float(box_area)
            ratio_img = mask_area / float(img_area) if img_area > 0 else 0.0

            if ratio_box > NEGATIVE_OVERSEG_BOX_RATIO or ratio_img > NEGATIVE_OVERSEG_IMG_RATIO:
                mask = bbox_to_mask(bbox, union_mask.shape)

            union_mask |= mask

        mask_dict[img_id] = union_mask
    except Exception as e:
        print(f"⚠️  Mask 실패 ({img_id}): {e}")

print()
print(f"✅ Mask 생성 완료: {len(mask_dict)}/{len(bbox_dict)}")


# ===== 11. 제출 파일 생성 =====
print()
print("="*80)
print("📄 제출 파일 생성 중...")
print("="*80)

submission_data = []

for img_id in sorted(test_images.keys()):
    if img_id in mask_dict:
        mask = mask_dict[img_id]
        rle_string = rle_encode(mask)
    else:
        rle_string = ""

    submission_data.append({
        "ID": img_id,
        "Label": rle_string
    })

submission_df = pd.DataFrame(submission_data)
submission_path = OUTPUT_DIR / "submission_challenge1_visual_focus.csv"
submission_df.to_csv(submission_path, index=False)

print(f"✅ 제출 파일 저장: {submission_path}")
print(f"   총 {len(submission_df)}개 예측")
print()
print("파일 미리보기:")
print(submission_df.head())


# ===== 12. 결과 시각화 =====
print()
print("="*80)
print("📊 결과 시각화 중...")
print("="*80)

import random
sample_ids = random.sample(list(mask_dict.keys()), min(4, len(mask_dict)))

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for idx, img_id in enumerate(sample_ids):
    # 원본
    axes[0, idx].imshow(test_images[img_id])
    axes[0, idx].set_title(f"{img_id}\n{queries[img_id][:30]}...", fontsize=8)
    axes[0, idx].axis('off')

    # 마스크
    axes[1, idx].imshow(mask_dict[img_id], cmap='gray')
    axes[1, idx].set_title("Mask", fontsize=8)
    axes[1, idx].axis('off')

plt.tight_layout()
viz_path = OUTPUT_DIR / "visualization_final.png"
plt.savefig(viz_path, dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ 시각화 저장: {viz_path}")


# ===== 13. 최종 요약 =====
print()
print("="*80)
print("✅ 모든 작업 완료!")
print("="*80)
print(f"📁 출력: {OUTPUT_DIR}")
print(f"📄 제출 파일: {submission_path}")
print(f"📊 시각화: {viz_path}")
print()
print("⚠️  제출 시 기록:")
print(f"   - Seed: {SEED}")
print("   - do_sample: False ✅")
print()
print("📊 결과:")
print(f"   - 전체: {len(test_images)}장")
print(f"   - BBox 성공: {len(bbox_dict)}장")
print(f"   - Fallback: {fallback_count}장")
print(f"   - Mask 생성: {len(mask_dict)}장")
print()
print("🎉 submission_challenge1_visual_focus.csv를 제출하세요!")


🚀 경진대회 문제 1-1
PyTorch: 2.6.0+cu118
CUDA: True
✅ Seed fixed: 42

📁 Dataset: /home/jjs2403/2026_bootcamp_02/dataset/challenge_datasets/challenge1
📁 Output: /home/bootcamp2026_01/challenge1_outputs

📂 데이터셋 로딩 중...


이미지 로딩:   0%|          | 0/129 [00:00<?, ?it/s]

✅ 이미지: 129장
✅ 쿼리: 129개

🤖 MLLM 모델 로딩 중...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ MLLM 로딩 완료

🎯 SAM2.1 모델 로딩 중...
✅ SAM2.1 로딩 완료

📍 BBox 예측 시작 (Visual Focus Refinement)...


BBox 예측:   0%|          | 0/129 [00:00<?, ?it/s]


✅ BBox 예측 완료: 129/129
   Fallback 사용: 2개
   Plural debug saved: /home/bootcamp2026_01/challenge1_outputs/plural_debug.csv (rows: 41)

🎭 Mask 생성 시작 (multimask)...


Mask 생성:   0%|          | 0/129 [00:00<?, ?it/s]

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Visualize all masks from a submission CSV (RLE) in batches.

Example:
  python visualize_submission_masks.py \
    --submission /home/<user>/challenge1_outputs/submission_challenge1_final.csv \
    --dataset /home/jjs2403/2026_bootcamp_02/dataset/challenge_datasets/challenge1/test \
    --batch-size 4
"""

import argparse
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt


def load_test_images(test_dir: Path) -> dict:
    patterns = ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"]
    image_files = []
    for pat in patterns:
        image_files.extend(sorted(test_dir.glob(pat)))
    images = {}
    for img_path in image_files:
        img_id = img_path.stem
        images[img_id] = Image.open(img_path).convert("RGB")
    return images


def rle_decode(rle: str, shape: tuple) -> np.ndarray:
    if not rle:
        return np.zeros(shape, dtype=bool)
    s = list(map(int, rle.split()))
    starts = s[0::2]
    lengths = s[1::2]
    starts = [x - 1 for x in starts]
    flat = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for st, ln in zip(starts, lengths):
        flat[st : st + ln] = 1
    return flat.reshape(shape).astype(bool)


def main() -> None:
    default_submission = Path.home() / "challenge1_outputs" / "submission_challenge1_visual_focus.csv"
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--submission",
        default=str(default_submission),
        help="Path to submission CSV with columns: ID, Label",
    )
    parser.add_argument(
        "--dataset",
        default="/home/jjs2403/2026_bootcamp_02/dataset/challenge_datasets/challenge1/test",
        help="Path to test images folder",
    )
    parser.add_argument("--batch-size", type=int, default=4)
    # Jupyter passes extra args (e.g., -f <kernel.json>), so ignore unknowns.
    args, _ = parser.parse_known_args()

    submission_path = Path(args.submission)
    test_dir = Path(args.dataset)

    if not submission_path.exists():
        raise FileNotFoundError(f"Submission not found: {submission_path}")
    if not test_dir.exists():
        raise FileNotFoundError(f"Dataset not found: {test_dir}")

    images = load_test_images(test_dir)
    df = pd.read_csv(submission_path)

    id_col = next((c for c in df.columns if c.lower() in ("id", "image_id", "filename")), df.columns[0])
    label_col = next((c for c in df.columns if c.lower() in ("label", "rle")), df.columns[1])

    mask_dict = {}
    for _, row in df.iterrows():
        img_id = str(row[id_col]).replace(".jpg", "").replace(".png", "")
        rle = "" if pd.isna(row[label_col]) else str(row[label_col])
        if img_id not in images:
            continue
        w, h = images[img_id].size
        mask_dict[img_id] = rle_decode(rle, (h, w))

    print("=" * 80)
    print("Visualization (All Results)...")
    print("=" * 80)

    all_ids = sorted(list(mask_dict.keys()))
    batch_size = max(1, args.batch_size)

    for i in range(0, len(all_ids), batch_size):
        batch_ids = all_ids[i : i + batch_size]
        curr_batch_len = len(batch_ids)

        fig, axes = plt.subplots(2, curr_batch_len, figsize=(4 * curr_batch_len, 6))
        if curr_batch_len == 1:
            axes = axes.reshape(2, 1)

        for idx, img_id in enumerate(batch_ids):
            axes[0, idx].imshow(images[img_id])
            axes[0, idx].set_title(f"{img_id}", fontsize=10)
            axes[0, idx].axis("off")

            axes[1, idx].imshow(mask_dict[img_id], cmap="gray")
            axes[1, idx].set_title("Mask", fontsize=10)
            axes[1, idx].axis("off")

        plt.tight_layout()
        plt.show()

    
if __name__ == "__main__":
    main()


Visualization (All Results)...


In [ ]:
# ===== BBox Visualization (Inline Grid) =====
import matplotlib.patches as patches

def _normalize_bboxes(bbox_val):
    if bbox_val is None:
        return []
    # dict style
    if isinstance(bbox_val, dict) and 'bbox_2d' in bbox_val:
        bbox_val = bbox_val['bbox_2d']
    # list of boxes or single box
    boxes = []
    if isinstance(bbox_val, (list, tuple)):
        # nested list: [[x1,y1,x2,y2], ...]
        if len(bbox_val) > 0 and isinstance(bbox_val[0], (list, tuple)):
            for b in bbox_val:
                if isinstance(b, (list, tuple)) and len(b) >= 4:
                    x1, y1, x2, y2 = b[:4]
                    boxes.append([int(x1), int(y1), int(x2), int(y2)])
        # flat list: [x1,y1,x2,y2]
        elif len(bbox_val) >= 4:
            x1, y1, x2, y2 = bbox_val[:4]
            boxes.append([int(x1), int(y1), int(x2), int(y2)])
    return boxes

cols = 4
max_viz = 129  # change if needed
viz_ids = list(bbox_dict.keys())[:max_viz]

# filter valid
valid = []
for img_id in viz_ids:
    bbs = _normalize_bboxes(bbox_dict.get(img_id))
    if not bbs:
        continue
    valid.append((img_id, bbs))

if not valid:
    print('No valid bboxes to display.')
else:
    rows = (len(valid) + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows))
    if rows == 1:
        axes = [axes]
    axes = [ax for row in axes for ax in (row if isinstance(row, (list, tuple, np.ndarray)) else [row])]

    for idx, (img_id, bboxes) in enumerate(valid):
        ax = axes[idx]
        image = test_images[img_id]
        ax.imshow(image)
        for b in bboxes:
            x1, y1, x2, y2 = b
            rect = patches.Rectangle((x1, y1), (x2 - x1 + 1), (y2 - y1 + 1),
                                     linewidth=2.0, edgecolor='lime', facecolor='none')
            ax.add_patch(rect)
        ax.set_title(f'{img_id} (boxes: {len(bboxes)})', fontsize=10)
        ax.axis('off')

    # turn off unused axes
    for j in range(len(valid), len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()
    print(f'✅ Displayed {len(valid)} bbox visualizations inline in a grid.')
